In [1]:
import tensorflow as tf 

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [38]:
data_x, data_y = [], []
num_id = 9
img_h=250
img_w=250
img_c=3
img_length = 250*250*3

for i in range(num_id):
    part_x, part_y = [], []
    folder = 'pixiv/quar_'+str(i)
    for img_file in os.listdir(folder):
#         print(folder+'/'+img_file)
        img = np.array(Image.open(folder+'/'+img_file))
#         img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
        part_x.append(img/255.0)
        
        label = [0 for j in range(0, num_id)]
        label[i]=1
        part_y.append(label)   
        if(len(part_x)>=590):
            break
    print(len(part_x))
    data_x.extend(part_x)
    data_y.extend(part_y)
    print(folder+' is completed.')

print(len(data_x))
print(len(data_y))

590
pixiv/quar_0 is completed.
590
pixiv/quar_1 is completed.
590
pixiv/quar_2 is completed.
590
pixiv/quar_3 is completed.
590
pixiv/quar_4 is completed.
590
pixiv/quar_5 is completed.
590
pixiv/quar_6 is completed.
590
pixiv/quar_7 is completed.
590
pixiv/quar_8 is completed.
5310
5310


In [39]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:10])
print(data_y[:10])

[[[[ 0.53725493  0.46666667  0.51372552]
   [ 0.54901963  0.40000001  0.4509804 ]
   [ 0.58039218  0.32156864  0.36862746]
   ..., 
   [ 0.57254905  0.7019608   0.83137256]
   [ 0.54901963  0.68235296  0.82745099]
   [ 0.56078434  0.69411767  0.84313726]]

  [[ 0.51764709  0.38039216  0.45882353]
   [ 0.56862748  0.35686275  0.44705883]
   [ 0.63921571  0.32941177  0.41176471]
   ..., 
   [ 0.49803922  0.63529414  0.75294119]
   [ 0.57254905  0.70980394  0.83529413]
   [ 0.58431375  0.72549021  0.86666667]]

  [[ 0.53333336  0.33333334  0.45490196]
   [ 0.57254905  0.30980393  0.43921569]
   [ 0.65490198  0.29803923  0.41960785]
   ..., 
   [ 0.01568628  0.16078432  0.26274511]
   [ 0.37254903  0.51764709  0.62352943]
   [ 0.50588238  0.65098041  0.7647059 ]]

  ..., 
  [[ 0.49803922  0.68235296  0.85490197]
   [ 0.49803922  0.68235296  0.85490197]
   [ 0.49803922  0.68235296  0.85490197]
   ..., 
   [ 0.35686275  0.52156866  0.61960787]
   [ 0.52549022  0.65098041  0.74117649]
   [ 0.

In [40]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

5310
4779
4779
531


In [41]:
train_x_list, train_y_list = [], []
valid_x_list, valid_y_list = [], []

n=int(len(data_x)/10)
for i in range(0, 9):
#     train_x_list.append([train_x[i] for i in range(0, i*n)+range((i+1)*n), len(train_x)])
    cur_x = [train_x[i] for i in range(0, i*n)]
    cur_x.extend([train_x[i] for i in range((i+1)*n, len(train_x))])
    cur_y = [train_y[i] for i in range(0, i*n)]
    cur_y.extend([train_y[i] for i in range((i+1)*n, len(train_x))])
    print(len(cur_x))
#     print(len(cur_y))
    
    train_x_list.append(cur_x)
    train_y_list.append(cur_y)
    
    valid_x_list.append([train_x[i] for i in range(i*n, (i+1)*n)])
    valid_y_list.append([train_y[i] for i in range(i*n, (i+1)*n)])

print(len(train_x_list))

4248
4248
4248
4248
4248
4248
4248
4248
4248
9


In [57]:
#3-layer
graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    labels = tf.placeholder('float', [None, num_id])            #
    inputs = tf.placeholder('float', [None, img_h, img_w, img_c])                        #输入的数据占位符
#     print(x.shape)
#     print(y_actual.shape)

    def weight_variable(shape): #定义一个函数，用于初始化所有的权值 W
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape): #定义一个函数，用于初始化所有的偏置项 b
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W): #定义一个函数，用于构建卷积层
        return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')

    def max_pool(x): #定义一个函数，用于构建池化层
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #构建网络
#     x_image = tf.reshape(inputs, [-1,250,250,3])         #转换输入数据shape,以便于用于网络中
    W_conv1 = weight_variable([5, 5, 3, 32])      
    b_conv1 = bias_variable([32])       
    h_conv1 = tf.nn.relu(conv2d(inputs, W_conv1) + b_conv1)     #第一个卷积层
    h_pool1 = max_pool(h_conv1)                                  #第一个池化层
    print(h_pool1.shape)#(?, 124, 124, 32)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
    h_pool2 = max_pool(h_conv2)                                   #第二个池化层
    print(h_pool2.shape)#(?, 30, 30, 64)
    
    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)      #第二个卷积层
    h_pool3 = max_pool(h_conv3)                                   #第二个池化层
    print(h_pool3.shape)#(?, 30, 30, 64)    

    # 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
    W_fc1 = weight_variable([4 * 4 * 128, 512])
    b_fc1 = bias_variable([512])
    h_pool3_flat = tf.reshape(h_pool3, [-1, 4*4*128])              #reshape成向量
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)    #第一个全连接层

    keep_prob = tf.placeholder("float") 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

    W_fc2 = weight_variable([512, num_id])
    b_fc2 = bias_variable([num_id])
#     y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层
    y_predict=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#     loss = -tf.reduce_sum(labels*tf.log(y_predict))     #交叉熵
#     optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)    #梯度下降法
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels = labels))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(labels,1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))    #精确度计算

(?, 63, 63, 32)
(?, 16, 16, 64)
(?, 4, 4, 128)


In [58]:
batch_start = 0
batch_size = 256
training_steps = 101

for i in range(0, 9):
    train_x = train_x_list[i]
    train_y = train_y_list[i]
#     config = tf.ConfigProto(allow_soft_placement=True)
#     config.gpu_options.allocator_type = 'BFC'
#     config.gpu_options.per_process_gpu_memory_fraction = 0.8
    with tf.Session(graph=graph) as session:
    #     saver.restore(session, tf.train.latest_checkpoint('lstm_check'))
        tf.global_variables_initializer().run()
        print('Initialized', i)
        for step in range(training_steps): 
            if(batch_start+batch_size>len(train_x)):
                order = [i for i in range(len(train_x))]
                random.shuffle(order)
                train_x = [train_x[i] for i in order]
                train_y = [train_y[i] for i in order]
                batch_start = 0        
            feed_x = train_x[batch_start:batch_start+batch_size]
            feed_y = train_y[batch_start:batch_start+batch_size]
            feed_dict = {inputs: feed_x, labels: feed_y, keep_prob:0.6}
            _, l, predictions, accur = session.run([optimizer, loss, labels, accuracy], feed_dict = feed_dict)
            if (step % 10 == 0):
                print('Loss at step %d: %f' % (step, l), ' accuarcy is ', accur)
    #             saver.save(session, 'lstm_check/my-model-1.ckpt', global_step=step+1000)
        valid_loss, valid_accur = session.run([loss,accuracy], feed_dict={inputs: valid_x_list[i], labels:  valid_y_list[i], keep_prob:1.0})
        print('==valid loss, ', valid_loss, ' valid accur is ', valid_accur)
        test_loss, test_accur = session.run([loss,accuracy], feed_dict={inputs: test_x, labels: test_y, keep_prob:1.0})
        print('--test loss, ', test_loss, ' test accur is ', test_accur)

Initialized 0
Loss at step 0: 27.572247  accuarcy is  0.144531
Loss at step 10: 2.691183  accuarcy is  0.179688
Loss at step 20: 1.692639  accuarcy is  0.375
Loss at step 30: 1.252351  accuarcy is  0.617188
Loss at step 40: 0.781725  accuarcy is  0.78125
Loss at step 50: 0.449732  accuarcy is  0.886719
Loss at step 60: 0.239948  accuarcy is  0.945312
Loss at step 70: 0.130373  accuarcy is  0.972656
Loss at step 80: 0.077832  accuarcy is  0.988281
Loss at step 90: 0.049984  accuarcy is  0.996094
Loss at step 100: 0.030665  accuarcy is  0.992188
==valid loss,  2.69191  valid accur is  0.3258
--test loss,  2.42775  test accur is  0.323917
Initialized 1
Loss at step 0: 40.181953  accuarcy is  0.105469
Loss at step 10: 2.812297  accuarcy is  0.148438


KeyboardInterrupt: 

9 - 0.35

5 - 0.3 - 0.4

3 - 0.55+